In [1]:
import cv2 as cv
import numpy as np
#import util

In [2]:
plate_image_file = 'images/candidate_plate.jpg'
candidate_plate_image = cv.imread(plate_image_file)

In [3]:
# 1. 图片预处理：灰度+二值化
# gray_image = cv.cvtColor(candidate_plate_image, cv.COLOR_BGR2GRAY)
gray_image = cv.cvtColor(candidate_plate_image, cv.COLOR_BGR2GRAY)
is_success, binary_image = cv.threshold(gray_image, 0, 255, cv.THRESH_OTSU)

In [4]:
# 2. 向内缩进，去除外边框
# 经验值
offset_X = 3
offset_Y = 5
# 切片提取内嵌区域
offset_region = binary_image[offset_Y:-offset_Y, offset_X:-offset_X]
# 生成工作区域
working_region = offset_region

In [5]:
# 3. 对车牌区域进行等值线找区域（要先处理下汉字-模糊化）
# 经验值：汉字区域占整体的 1/8
chinese_char_max_width = working_region.shape[1] // 8
# 提取汉字区域
chinese_char_region = working_region[:, 0:chinese_char_max_width]
# 对汉字区域进行模糊处理
cv.GaussianBlur(chinese_char_region, (9, 9), 0, dst=chinese_char_region)
# 对整个区域找轮廓==等值线
char_contours, _ = cv.findContours(working_region, cv.RETR_CCOMP, cv.CHAIN_APPROX_NONE)
# cv.drawContours(candidate_plate_image, char_contours, -1, (0,0,255))
# cv.imshow('tmp', candidate_plate_image)
# cv.waitKey()
# cv.destroyAllWindows()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

array([[[154,  78,  56],
        [156,  78,  55],
        [155,  76,  49],
        ...,
        [142,  76,  41],
        [129,  68,  34],
        [147,  89,  54]],

       [[153,  77,  55],
        [153,  77,  54],
        [155,  76,  49],
        ...,
        [149,  83,  48],
        [138,  75,  41],
        [153,  95,  60]],

       [[151,  75,  56],
        [150,  75,  53],
        [153,  76,  50],
        ...,
        [159,  91,  56],
        [148,  84,  49],
        [162, 102,  66]],

       ...,

       [[155,  65,  54],
        [164,  75,  61],
        [167,  77,  66],
        ...,
        [146,  92,  67],
        [168, 114,  89],
        [196, 145, 119]],

       [[153,  67,  55],
        [161,  75,  63],
        [166,  77,  66],
        ...,
        [145,  91,  66],
        [165, 114,  88],
        [195, 144, 118]],

       [[152,  68,  56],
        [159,  75,  63],
        [163,  77,  65],
        ...,
        [144,  90,  65],
        [164, 113,  87],
        [191, 142, 116]]

32

In [7]:
# 4. 过滤不合适的轮廓（等值线框）
# 经验值
CHAR_MIN_WIDTH = working_region.shape[1] // 40
CHAR_MIN_HEIGHT = working_region.shape[0] * 7 // 10

In [8]:
# 5. 逐个遍历所有候选的字符区域轮廓==等值线框，按照大小进行过滤
valid_char_regions = []
for i in np.arange(len(char_contours)):
    x, y, w, h = cv.boundingRect(char_contours[i])
    if w >= CHAR_MIN_WIDTH and h >= CHAR_MIN_HEIGHT: 
        # 将字符区域的中心点x的坐标 和 字符区域 作为一个元组，放入 valid_char_regions 列表
        valid_char_regions.append((x, offset_region[y:y+h, x:x+w]))

In [10]:
# 6. 按找区域的x坐标进行排序，并返回字符列表
sorted_regions = sorted(valid_char_regions, key=lambda region:region[0])
# valid_char_regions
# sorted_regions
candidate_char_images = []
for i in np.arange(len(sorted_regions)):
    candidate_char_images.append(sorted_regions[i][1])

In [11]:
for candidate_char_image in candidate_char_images:
    cv.imshow('tmp', candidate_char_image)
    cv.waitKey()
cv.destroyAllWindows()

32

32

32

32

32

32

32

32